<a href="https://colab.research.google.com/github/vjumpkung/vjump-sd-webui-colab/blob/main/Colab_SDXL_LoRA_Trainer_by_vjumpkung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **kohya-ss GUI v24.3.0 Colab Version**

- แนะนำ Colab Pro T4 High Ram หรือ สูงกว่าในการ Train LoRA

- Colab Pro GPU T4 High Ram สามารถรันได้แค่ fp16 เท่านั้น หากต้องการใช้ BF16 ให้ใช้ GPU L4 ขึ้นไป

- การเลือกตำแหน่งของ path ให้คลิ๊กขวาแล้วกด copy path แล้ววางในแต่ละส่วน

- ตำแหน่งของ model จะอยู่ที่ folder model

- หากต้องการหยุดโปรแกรม (kohya-ss gui) ให้กด Runtime -> Interrupt Execution

## รายชื่อ Model ที่มีให้ Download

### Stable Diffusion 1.5

- Stable Diffusion 1.5 (Base Model)
- AnyLoRA (สำหรับเทรน Anime)
- ChilloutMix (สำหรับ Realistic หน้าเอเชีย)

### SDXL

- SDXL V1.0 (Base Model)
- Juggernaut XL XI (สำหรับ Realistic)
- animagine-xl-3.1 (สำหรับ Anime)

### SDXL Fine-Tuned

- Pony Diffusion V6 XL (Pony)
- Illustrious-XL-v0.1 (Illustrious)

### SD3

- Stable Diffusion 3 Medium

### SD3.5

- Stable Diffusion 3.5 Large
- Stable Diffusion 3.5 Medium

### Flux

- Flux.1 Dev (FP16)
- Flux.1 Dev (FP8)

## Initialization (กด run อันนี้ก่อนเสมอ)

In [ ]:
import os
import sys
from google.colab import drive
from IPython.display import clear_output
import requests
from google.colab import drive
drive.mount('/content/drive')
!apt-get -y install -qq aria2 &> /dev/null
!git clone --recurse-submodules https://github.com/vjumpkung/kohya_ss.git
!uv pip install --system -r https://gist.githubusercontent.com/vjumpkung/de08e1032b14734caa7177d26159fd74/raw/e5220d037a9d0f8082ab44aef9de2b56fcce791e/colab_kohya_req.txt
!wget https://raw.githubusercontent.com/vjumpkung/vjump-runpod-notebooks-and-script/refs/heads/main/ui/google_drive_download.py
!mkdir -p ./model/checkpoints/ ./model/unet/ ./model/clip/ ./model/vae/ ./lora_project/

In [ ]:
!wget https://raw.githubusercontent.com/vjumpkung/vjump-runpod-notebooks-and-script/refs/heads/main/colab_kohya_ss/main.py
exec("from main import *")

## (ไม่บังคับ) ใส่ CivitAI API Key หรือ Huggingface token

In [ ]:
exec("setup()")

### Download Dataset จาก Google Drive/Huggingface

- แนะนำให้ upload ลง google drive ให้เป็นไฟล์ .zip

In [ ]:
exec("download_dataset()")

## เลือก Pretrained Model ที่ต้องการ Download (checkpoint)

In [ ]:
exec("select_pretrained_model()")

## เลือก Clip, VAE (สำหรับ Flux และ SD3)

In [ ]:
exec("select_clip_vae_model()")

## เริ่มโปรแกรม Kohya-SS GUI

In [ ]:
exec("launch_kohya_ss()")

## หากพบปัญหาสามารถแจ้งได้ที่

facebook : https://www.facebook.com/chanrich.pisitjing/